# Training a (Tail) Padded Chatbot

In this model we will take reddit comments and use them for training a RNN using tensorflow 2.0 to create a chatbot!

Some resources can be found here:<br/>
<https://www.tensorflow.org/tutorials/text/nmt_with_attention>

In [13]:
import collections, io, os, re, time, unicodedata
import numpy as np, tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [5]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(s):
    """
    convert to unicode
    make punctuation tokens
    add start and stop tokens
    """
    s = unicode_to_ascii(s.strip())
    s = re.sub(r"([?.!,])", r" \1 ", s)
    s = '<start> ' + s.rstrip().strip() + ' <end>'
    return s

# zip up data into format of 
# [
#  [comment, reply],
#  ...
# ]
def create_dataset(from_data_fname, to_data_fname, dirname, *, max_rows=None):
    from_data_path = os.path.join(dirname, from_data_fname)
    to_data_path = os.path.join(dirname, to_data_fname)
    
    from_lines = io.open(from_data_path, encoding='UTF-8').read().strip().split('\n')
    to_lines = io.open(to_data_path, encoding='UTF-8').read().strip().split('\n')
    assert(len(from_lines) == len(to_lines))
    from_lines = [preprocess_sentence(l) for l in from_lines[:max_rows]]
    to_lines = [preprocess_sentence(l) for l in to_lines[:max_rows]]
    return from_lines, to_lines

In [6]:
test_comment = "Double just means that you're sharing an individual room with a roommate. You're still in a suite (three rooms, six total people)."
test_reply = "I thought bluestone dorms were two rooms with four total people?"
print(preprocess_sentence(test_comment))
print(preprocess_sentence(test_reply))

<start> Double just means that you're sharing an individual room with a roommate .  You're still in a suite (three rooms ,  six total people) . <end>
<start> I thought bluestone dorms were two rooms with four total people ? <end>


In [10]:
def max_length(tensor):
    return max(len(t) for t in tensor)

def tokenize(inputs):
    tokenizer = keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(inputs)
    
    tensor = tokenizer.texts_to_sequences(inputs)
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, tokenizer

def load_dataset(path='data/formatted', num_rows_to_use=None):
    from_lines, to_lines = create_dataset('train_from', 'train_to', path, max_rows=num_rows_to_use)
    input_tensor, input_tokenizer = tokenize(from_lines)
    target_tensor, target_tokenizer = tokenize(to_lines)
    return input_tensor, target_tensor, input_tokenizer, target_tokenizer

In [11]:
path_to_data = os.path.join('data', 'formatted')
input_tensor, target_tensor, input_tokenizer, target_tokenizer = load_dataset(path_to_data)
max_length_input, max_length_target = max_length(input_tensor), max_length(target_tensor)

In [14]:
# Tomorrow combine both training and test files :eyeroll:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

596 596 150 150


In [15]:
def convert(tokenizer, tensor):
    for t in tensor:
        if t!=0:
            print("%d ----> %s" % (t, tokenizer.index_word[t]))

print ("Input Comment; index to word mapping")
convert(input_tokenizer, input_tensor_train[0])
print ()
print ("Target Reply; index to word mapping")
convert(target_tokenizer, target_tensor_train[0])

Input Comment; index to word mapping
2 ----> <start>
337 ----> unless
13 ----> it
28 ----> was
2457 ----> gudako
5 ----> ,
134 ----> which
12 ----> is
48 ----> about
37 ----> as
280 ----> likely
37 ----> as
6 ----> a
2458 ----> bugblatter
824 ----> beast
16 ----> of
2459 ----> traal
2460 ----> slipping
17 ----> in
6 ----> a
2461 ----> request
426 ----> form
7 ----> to
6 ----> a
2462 ----> vogon
7 ----> to
2463 ----> consume
222 ----> its
2464 ----> grandmother
1 ----> .
3 ----> <end>

Target Reply; index to word mapping
2 ----> <start>
2112 ----> &gt;unless
10 ----> it
28 ----> was
2113 ----> gudako
14 ----> new_line
14 ----> new_line
2114 ----> gudao
16 ----> is
2115 ----> 'officially'
2116 ----> dws
2117 ----> protag
17 ----> in
41 ----> all
2118 ----> fate/grand
289 ----> order
2119 ----> animations
1 ----> .
2120 ----> riyodako
2121 ----> overused
349 ----> rule
2122 ----> breaker
31 ----> so
52 ----> she
141 ----> can't
23 ----> be
2123 ----> casted
1 ----> .
3 ----> <end>


In [16]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = BUFFER_SIZE // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_input_size, vocab_target_size = len(input_tokenizer.word_index)+1, len(target_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

ex_input_batch, ex_target_batch = next(iter(dataset))
print(ex_input_batch.shape, ex_target_batch.shape)

(64, 62) (64, 56)


In [17]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))


In [18]:
encoder = Encoder(vocab_input_size, embedding_dim, units, BATCH_SIZE)

sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(ex_input_batch, sample_hidden)

In [19]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights


In [20]:
attention_layer = BahdanauAttention(10)
attn_result, attn_weights = attention_layer(sample_hidden, sample_output)

In [21]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights


In [22]:
decoder = Decoder(vocab_target_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)), sample_hidden, sample_output)

In [23]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)


In [24]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)


In [25]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([target_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss


In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
